<a href="https://colab.research.google.com/github/sumanyurosha/tensorflow-specialization/blob/master/Hands-on%20ML/chapter16/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Downloading the IMDb Movie Reviews Dataset**

In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

In [3]:
dataset, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples

In [4]:
train_size

25000

# **Writing the Preprocessing Pipeline**

In [5]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, b"<br\\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

# **Creating a Vocabulary**

In [12]:
from collections import Counter

vocabulary = Counter()

for X_batch, y_batch in dataset["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [15]:
len(vocabulary.most_common())

54104

In [17]:
vocab_size = 10000
truncated_vocabulary = [word for word, count
                        in vocabulary.most_common()[:vocab_size]]

# **Creating a Look up Table for converting our Reviews to Sequence of integers**

In [18]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 100
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [19]:
table.lookup(tf.constant([b"This movie was fataaaastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10039]])>

# **Doind the Batching and Prefetching**

In [20]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_dataset = dataset["train"].batch(32).map(preprocess)
train_dataset = train_dataset.map(encode_words).prefetch(1)

# **Creating and Training the Model**

In [21]:
embed_dim = 128

model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_dim, input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

history = model.fit(train_dataset, epochs=5)

Epoch 1/5
782/782 [==============================] - 37s 48ms/step - loss: 0.6015 - accuracy: 0.6416
Epoch 2/5
782/782 [==============================] - 37s 47ms/step - loss: 0.3859 - accuracy: 0.8293
Epoch 3/5
782/782 [==============================] - 37s 47ms/step - loss: 0.2509 - accuracy: 0.9036
Epoch 4/5
782/782 [==============================] - 37s 47ms/step - loss: 0.1639 - accuracy: 0.9420
Epoch 5/5
782/782 [==============================] - 36s 46ms/step - loss: 0.1367 - accuracy: 0.9508


# **Using Masking**

In [23]:
K = keras.backend

inputs = keras.layers.Input(shape=[None])
mask = keras.layers.Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs)
z = keras.layers.Embedding(vocab_size + num_oov_buckets, embed_dim)(inputs)
z = keras.layers.GRU(128, return_sequences=True)(z, mask=mask)
z = keras.layers.GRU(128)(z, mask=mask)
outputs = keras.layers.Dense(1, activation="sigmoid")(z)

model = keras.Model(inputs=[inputs], outputs=[outputs])

In [24]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

history = model.fit(train_dataset, epochs=5)

Epoch 1/5
782/782 [==============================] - 41s 52ms/step - loss: 0.5577 - accuracy: 0.6995
Epoch 2/5
782/782 [==============================] - 40s 52ms/step - loss: 0.3675 - accuracy: 0.8381
Epoch 3/5
782/782 [==============================] - 41s 52ms/step - loss: 0.2400 - accuracy: 0.9071
Epoch 4/5
782/782 [==============================] - 40s 51ms/step - loss: 0.1528 - accuracy: 0.9440
Epoch 5/5
782/782 [==============================] - 40s 52ms/step - loss: 0.1150 - accuracy: 0.9582


# **Reusing Pretrained Embedding**

In [29]:
dataset, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_dataset = dataset["train"].batch(32).prefetch(1)

In [31]:
import tensorflow_hub as hub

model = keras.models.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1", 
                   dtype=tf.string, input_shape=[], output_shape=[50]),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(train_dataset, epochs=5)

Epoch 1/5
782/782 [==============================] - 7s 9ms/step - loss: 0.5504 - accuracy: 0.7232
Epoch 2/5
782/782 [==============================] - 7s 9ms/step - loss: 0.5145 - accuracy: 0.7474
Epoch 3/5
782/782 [==============================] - 7s 9ms/step - loss: 0.5090 - accuracy: 0.7503
Epoch 4/5
782/782 [==============================] - 7s 9ms/step - loss: 0.5053 - accuracy: 0.7533
Epoch 5/5
782/782 [==============================] - 7s 9ms/step - loss: 0.5023 - accuracy: 0.7559
